CIFAR-10 is loaded using the keras datasets interface. The per-pixel mean is subtracted as preprocessing. The code in this notebook is based on the keras example found here: https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py

In [1]:
import os
import numpy as np
import keras
from keras.datasets import cifar10

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

Using TensorFlow backend.


The model is created the same way as cifar10_resnet.py does. Checkpoints are saved in the "saved_models" directory. The model itself is a 29 layer deep version of resnet v2. More details can be found in cifar10_resnet.py.

In [ ]:
from keras.layers import *
from model import LogGaussMF
from resnet_backend import *


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', 1. * lr)
    return 1. * lr


# Prepare model model saving directory.
def get_callbacks(model_type):
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    model_name = "%s_model.{epoch:03d}.h5" % model_type
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filepath = os.path.join(save_dir, model_name)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(
        filepath=filepath,
        monitor='val_acc',
        verbose=1,
        save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(
        factor=np.sqrt(0.1),
        cooldown=0,
        patience=5,
        min_lr=0.5e-6)

    return [checkpoint, lr_reducer, lr_scheduler]


use_anfis = True

n = 6
depth = n * 9 + 2

# Model name, depth and version
model_type = 'Fuzzy-ResNet%dv%d' % (depth, 2)

inputs, features = resnet_backend_v2(
    input_shape=input_shape, 
    depth=depth)

memberships = LogGaussMF(10)(features)
rules = Lambda(lambda x: K.sum(x, axis=-1))(memberships)

if use_anfis:
    linear = Dense(10)(features)
    logits = Add()([rules, linear])
else:
    logits = rules

outputs = Activation("softmax")(logits)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=lr_schedule(0)),
    metrics=['accuracy'])

callbacks = get_callbacks(model_type)

W0415 15:59:57.918639 140052082915136 deprecation.py:506] From /home/ryan-desktop/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/training/moving_averages.py:210: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Learning rate:  0.001


Data augmentation is used to improve performance. The model took approximately 3-3.5 hours to train and reached an accuracy of 92.25%

In [ ]:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=32),
    steps_per_epoch=60000 // 32,
    validation_data=(x_test, y_test),
    epochs=200, 
    verbose=1, 
    workers=4,
    callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/200
Learning rate:  0.001
1875/1875 [==============================] - 125s 66ms/step - loss: 1.9894 - acc: 0.5016 - val_loss: 1.8723 - val_acc: 0.5225

Epoch 00001: val_acc improved from -inf to 0.52250, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/Fuzzy-ResNet56v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
1875/1875 [==============================] - 112s 60ms/step - loss: 1.3441 - acc: 0.6569 - val_loss: 1.4165 - val_acc: 0.6430

Epoch 00002: val_acc improved from 0.52250 to 0.64300, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/Fuzzy-ResNet56v2_model.002.h5
Epoch 3/200
Learning rate:  0.001
1875/1875 [==============================] - 113s 60ms/step - loss: 1.1451 - acc: 0.7127 - val_loss: 1.1956 - val_acc: 0.6962

Epoch 00003: val_acc improved from 0.64300 to 0.69620, saving model to /home/ryan-desktop/Documents/research-2019/cifar-10/saved_models/Fuzzy-ResNet56v2_model.003.h5
Epoch 4/200
Learning r

1875/1875 [==============================] - 110s 58ms/step - loss: 0.4581 - acc: 0.9094 - val_loss: 0.7472 - val_acc: 0.8291

Epoch 00071: val_acc did not improve from 0.88320
Epoch 72/200
Learning rate:  0.001
1875/1875 [==============================] - 109s 58ms/step - loss: 0.4592 - acc: 0.9094 - val_loss: 0.6549 - val_acc: 0.8609

Epoch 00072: val_acc did not improve from 0.88320
Epoch 73/200
Learning rate:  0.001
1875/1875 [==============================] - 109s 58ms/step - loss: 0.4547 - acc: 0.9107 - val_loss: 0.6496 - val_acc: 0.8605

Epoch 00073: val_acc did not improve from 0.88320
Epoch 74/200
Learning rate:  0.001
1875/1875 [==============================] - 110s 59ms/step - loss: 0.4514 - acc: 0.9134 - val_loss: 0.6437 - val_acc: 0.8579

Epoch 00074: val_acc did not improve from 0.88320
Epoch 75/200
Learning rate:  0.001
1875/1875 [==============================] - 109s 58ms/step - loss: 0.4561 - acc: 0.9107 - val_loss: 0.5644 - val_acc: 0.8804

Epoch 00075: val_acc did 

1875/1875 [==============================] - 109s 58ms/step - loss: 0.2035 - acc: 0.9768 - val_loss: 0.4000 - val_acc: 0.9228

Epoch 00106: val_acc did not improve from 0.92570
Epoch 107/200
Learning rate:  0.0001
1875/1875 [==============================] - 109s 58ms/step - loss: 0.1990 - acc: 0.9773 - val_loss: 0.3979 - val_acc: 0.9244

Epoch 00107: val_acc did not improve from 0.92570
Epoch 108/200
Learning rate:  0.0001
1875/1875 [==============================] - 110s 58ms/step - loss: 0.1976 - acc: 0.9781 - val_loss: 0.4124 - val_acc: 0.9217

Epoch 00108: val_acc did not improve from 0.92570
Epoch 109/200
Learning rate:  0.0001
1875/1875 [==============================] - 109s 58ms/step - loss: 0.1978 - acc: 0.9770 - val_loss: 0.3958 - val_acc: 0.9244

Epoch 00109: val_acc did not improve from 0.92570
Epoch 110/200
Learning rate:  0.0001
1875/1875 [==============================] - 108s 58ms/step - loss: 0.1929 - acc: 0.9786 - val_loss: 0.4010 - val_acc: 0.9236

Epoch 00110: val_

The modified version of resnet, replacing the classifier with the fuzzy classifier. It is trained with the same parameters as the normal version of resnet. It was able to achieve 92.56% accuracy, slightly better than the traditional version, but this may be due to random chance.